Now we parse the content of the JSON file with the QASM circuits

In [1]:
using ITensors
using IterTools

include("simulation/mpo_simulation.jl")
using .MPOSimulation
include("utils/utils.jl")
using .Utils
include("simulation/mps_simulation.jl")
using .MPSSimulation

In [25]:
# Simulation using MPS
json_filename = "subcircuits.json"  
circuits_data, expval_knitting, coefficients = parse_subcircuits(json_filename);

In [26]:
basis_gates = ["h", "rx", "ry", "rz", "rxx","rzz", "ryy", "cx"]
cutoff = 1e-12
maxdim = 10000
method = "densitymatrix" #currently this is not used

all_mpo_sequences, all_psis= mpo_sequence_apply_mps(circuits_data,basis_gates, cutoff, maxdim, method);

Completed applying MPO sequence for subcircuit B1.
Completed applying MPO sequence for subcircuit B2.
Completed applying MPO sequence for subcircuit B3.
Completed applying MPO sequence for subcircuit B4.
Completed applying MPO sequence for subcircuit B5.
Completed applying MPO sequence for subcircuit B6.
Completed applying MPO sequence for subcircuit B7.
Completed applying MPO sequence for subcircuit B8.
Completed applying MPO sequence for subcircuit B9.
Completed applying MPO sequence for subcircuit B10.
Completed applying MPO sequence for subcircuit B11.
Completed applying MPO sequence for subcircuit B12.
Completed applying MPO sequence for subcircuit B13.
Completed applying MPO sequence for subcircuit B14.
Completed applying MPO sequence for subcircuit B15.
Completed applying MPO sequence for subcircuit B16.
Completed applying MPO sequence for subcircuit B17.
Completed applying MPO sequence for subcircuit B18.
Completed applying MPO sequence for subcircuit B19.
Completed applying MP

In [27]:
labels = keys(all_psis)

# Collect site indices for each label (we take generic state from all_psis)
sites_per_label = Dict{String, Vector{Index{Int64}}}()
for label in labels
    sites_per_label[label] = siteinds(all_psis[label][1][1][1])
end
# Generate union of sites
sites_union = vcat(union(values(sites_per_label))...)

function create_mpo(op_name::String, site::Int, sites::Vector{Index{Int64}})
    os = OpSum()
    add!(os, op_name, site)
    return MPO(os, sites)
end

# Create the observable 
obs = sum([create_mpo("Z", i, sites_union) for i in 1:length(sites_union)]) # ZII + IZI + IIZ
# obs = [MPO(sites_AB, "Z")] # ZZZ

function compute_expval(psi::MPS, observable::MPO)
    # Compute ⟨psi| O |psi⟩
    expval = ITensor(1.0)
    for n in 1:length(psi)
        expval *= psi[n]
        expval *= observable[n]
        expval *= prime(dag(psi[n]))
    end
    return real(expval[1])
end

# Faster tensor product
function mps_tensor_product(mps_dict::Dict{String, MPS}, sites::Vector{Index{Int64}})
    total_length = sum(length(mps) for mps in values(mps_dict))
    psi = MPS(sites, "0")
    current_index = 1
    for label in keys(mps_dict)
        mps = mps_dict[label]
        for i in 1:length(mps)
            psi[current_index] = mps[i]
            current_index += 1
        end
    end
    return psi
end


# We reconstruct the expectation value (with MPS)
# Tr[Oρ'] = ∑ₓⁿ Tr[Oᴀₓ ⊗ Oᵦₓ ρ'] = ∑ₓⁿ ∑ₖ₌₁ᴷ ∑ⱼₐ₌₁ᵐₐ ∑ⱼᵦ₌₁ᵐᵦ αₖ pᴀⱼₐ pᵦⱼᵦ ⟨Oᴀₓ⟩ₖ,ⱼₐ ⟨Oᵦₓ⟩ₖ,ⱼᵦ
expval_tensors = 0.0
for (k, coeff) in enumerate(coefficients)
    temp = 0.0

    combinations = IterTools.product([all_psis[label][k] for label in labels]...)    
    for combination in combinations
        mps_dict = Dict{String, MPS}()
        prob = 1.0
        for (idx, label) in enumerate(labels)
            psi_label, p_label = combination[idx]
            mps_dict[label] = psi_label
            prob *= p_label
        end
        psi = mps_tensor_product(mps_dict, sites_union)
        temp += prob * compute_expval(psi, obs)
    end
    expval_tensors += coeff * temp
end

discrepancy = abs(round(100 * (expval_knitting - expval_tensors) / expval_knitting, digits=4))

println("""
==============================
• Expected Value Knitting: $(round(expval_knitting, digits=8))
• Expected Value Tensors:  $(round(expval_tensors, digits=8))
• Discrepancy            : $discrepancy%
==============================
""")


• Expected Value Knitting: 22.57930094
• Expected Value Tensors:  22.97923947
• Discrepancy            : 1.7713%



In [5]:
# Simulation using MPO
basis_gates = ["h", "rx", "ry", "rz", "rxx","rzz", "ryy", "cx"]
cutoff = 1e-12
maxdim = 10000
method = "densitymatrix" #currently this is not used

all_mpo_sequences, all_rhos = mpo_sequence_apply(circuits_data,basis_gates, cutoff, maxdim, method);

Completed applying MPO sequence for subcircuit B1.
Completed applying MPO sequence for subcircuit B2.
Completed applying MPO sequence for subcircuit B3.
Completed applying MPO sequence for subcircuit B4.
Completed applying MPO sequence for subcircuit B5.
Completed applying MPO sequence for subcircuit B6.
Completed applying MPO sequence for subcircuit A1.
Completed applying MPO sequence for subcircuit A2.
Completed applying MPO sequence for subcircuit A3.
Completed applying MPO sequence for subcircuit A4.
Completed applying MPO sequence for subcircuit A5.
Completed applying MPO sequence for subcircuit A6.


In [6]:
# Does not seem like a very efficient way, the old one is better

labels = keys(all_rhos)

# Collect site indices for each label (we take generic state from all_psis)
sites_per_label = Dict{String, Vector{Index{Int64}}}()
for label in labels
    sites_per_label[label] = Vector{Index{Int64}}()
    for inds in siteinds(all_rhos[label][2])
        push!(sites_per_label[label], inds[2])
    end
end


# Generate union of sites
sites_union = vcat(union(values(sites_per_label))...)

function create_mpo(op_name::String, site::Int, sites::Vector{Index{Int64}})
    os = OpSum()
    add!(os, op_name, site)
    return MPO(os, sites)
end

# Create the observable 
obs = sum([create_mpo("Z", i, sites_union) for i in 1:length(sites_union)]) # ZII + IZI + IIZ
# obs = [MPO(sites_AB, "Z")] # ZZZ

function mpo_tensor_product(mpo_dict::Dict{String, MPO}, sites::Vector{Index{Int64}})
    total_length = sum(length(mpo) for mpo in values(mpo_dict))
    rho = MPO(sites)
    current_index = 1
    for label in keys(mpo_dict)
        mpo = mpo_dict[label]
        for i in 1:length(mpo)
            rho[current_index] = mpo[i]
            current_index += 1
        end
    end
    return rho
end

# We reconstruct the expectation value (with MPO)
# Tr[Oρ'] = ∑ₓⁿ Tr[Oᴀₓ ⊗ Oᵦₓ ρ'] = ∑ₓⁿ ∑ₖ₌₁ᴷ ∑ⱼₐ₌₁ᵐₐ ∑ⱼᵦ₌₁ᵐᵦ αₖ pᴀⱼₐ pᵦⱼᵦ ⟨Oᴀₓ⟩ₖ,ⱼₐ ⟨Oᵦₓ⟩ₖ,ⱼᵦ
expval_tensors = 0.0

for (k, coeff) in enumerate(coefficients)
    temp = 0.0
    mpo_dict = Dict{String, MPO}()
    for label in labels
        mpo_dict[label] = all_rhos[label][k]
    end
    rho = mpo_tensor_product(mpo_dict, sites_union)
    expval_tensors += real(coeff * tr(apply(rho, obs)))
end

discrepancy = abs(round(100 * (expval_knitting - expval_tensors) / (expval_knitting), digits=4))

println("""
==============================
• Expected Value Knitting: $(round(expval_knitting, digits=8))
• Expected Value Tensors:  $(round(expval_tensors, digits=8))
• Discrepancy            : $discrepancy%
==============================
""")


• Expected Value Knitting: 7.65974649
• Expected Value Tensors:  7.65974649
• Discrepancy            : 0.0%

